# Cleaning 2016 County-Level Election Data
Isaac Kamber

Here we will walk through the steps needed to go from a shapefile of United States counties and a CSV of election results for the 21st century to a spatial object containing 2016 county-level election results which can then be written to a shapefile 

## Import the Required Packages
For the purposes of this process, we will need the following packages for the reasons commented beside them:

In [1]:
library(sp) #Working with spatial data
library(rgdal) #Working with spatial data
library(tidyverse) #Data wrangling and cleaning
library(tmap) #Data visualization

Warning message:
“package ‘rgdal’ was built under R version 3.4.4”rgdal: version: 1.3-9, (SVN revision 794)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.1.3, released 2017/20/01
 Path to GDAL shared files: /Users/isaackamber/Library/R/3.4/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: /Users/isaackamber/Library/R/3.4/library/rgdal/proj
 Linking to sp version: 1.3-1 
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.2.4  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.1       ✔ stringr 1.3.1  
✔ readr   1.1.1       ✔ forcats 0.3.0  
Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”Warning message:
“package ‘tibble’ was built under R version 3.4.4”Warning message:
“package ‘dplyr’ was built under R version 3.4.4”── Conflicts ──────────

## Load the Data into our Environment
Now that we have set up our environment with the requisite packages, it is time to load our data into our environment. The original datasource for the election results data can be found [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ&version=1.0) while the county shapefile can be found [here](https://www.kaggle.com/benhamner/2016-us-election).

In [2]:
election.results <- read.csv("countypres_2000-2016.csv")
us.counties <- readOGR("2016-us-election/county_shapefiles")

Warning message in readOGR("2016-us-election/county_shapefiles"):
“First layer cb_2014_us_county_5m read; multiple layers present in
/Users/isaackamber/Desktop/School Stuff/GIS 3/Final Project/2016-us-election/county_shapefiles, check layers with ogrListLayers()”

OGR data source with driver: ESRI Shapefile 
Source: "/Users/isaackamber/Desktop/School Stuff/GIS 3/Final Project/2016-us-election/county_shapefiles", layer: "cb_2014_us_county_5m"
with 3233 features
It has 9 fields
Integer64 fields read as strings:  ALAND AWATER 


Warning message in readOGR("2016-us-election/county_shapefiles"):
“Z-dimension discarded”

## Examining our Data
The data is now loaded into our environment and we can begin examining it. 

In [4]:
glimpse(election.results)

Observations: 50,528
Variables: 11
$ year           <int> 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, …
$ state          <fct> Alabama, Alabama, Alabama, Alabama, Alabama, Alabama, …
$ state_po       <fct> AL, AL, AL, AL, AL, AL, AL, AL, AL, AL, AL, AL, AL, AL…
$ county         <fct> Autauga, Autauga, Autauga, Autauga, Baldwin, Baldwin, …
$ FIPS           <int> 1001, 1001, 1001, 1001, 1003, 1003, 1003, 1003, 1005, …
$ office         <fct> President, President, President, President, President,…
$ candidate      <fct> Al Gore, George W. Bush, Ralph Nader, Other, Al Gore, …
$ party          <fct> democrat, republican, green, NA, democrat, republican,…
$ candidatevotes <int> 4942, 11993, 160, 113, 13997, 40872, 1033, 578, 5188, …
$ totalvotes     <int> 17208, 17208, 17208, 17208, 56480, 56480, 56480, 56480…
$ version        <int> 20181011, 20181011, 20181011, 20181011, 20181011, 2018…


In [5]:
glimpse(us.counties@data)

Observations: 3,233
Variables: 9
$ STATEFP  <fct> 01, 13, 19, 40, 42, 40, 31, 29, 32, 12, 05, 48, 49, 31, 42, …
$ COUNTYFP <fct> 059, 111, 109, 115, 115, 053, 029, 213, 510, 049, 023, 017, …
$ COUNTYNS <fct> 00161555, 00351094, 00465243, 01101845, 01213688, 01101814, …
$ AFFGEOID <fct> 0500000US01059, 0500000US13111, 0500000US19109, 0500000US401…
$ GEOID    <fct> 01059, 13111, 19109, 40115, 42115, 40053, 31029, 29213, 3251…
$ NAME     <fct> Franklin, Fannin, Kossuth, Ottawa, Susquehanna, Grant, Chase…
$ LSAD     <fct> 06, 06, 06, 06, 06, 06, 06, 06, 00, 06, 06, 06, 06, 06, 06, …
$ ALAND    <fct> 1641580723, 1002370118, 2519332669, 1219467406, 2132881515, …
$ AWATER   <fct> 32904833, 13560697, 4154722, 35708892, 22356541, 7021964, 79…


## Data Cleaning
We are first going to focus on the election data. We have more data than we need, as we only want to focus on 2016 for now. We also have the data in an extremely inconvenient format if we want to merge it with the counties shapefile later on. We will now work to remedy these issues. Rather than simply including total vote numbers, we want to examine voting percentages. 

In [6]:
#Select only 2016 election data
election.2016 <- election.results %>%
  dplyr::filter(year == 2016)

#Recast candidate from factor to string
election.2016$candidate <- as.character(election.2016$candidate)

#Remove other candidates
election.2016.2 <- election.2016 %>% 
  dplyr::filter(candidate %in% c("Hillary Clinton", "Donald Trump"))

hillary <- election.2016.2 %>%
  filter(candidate == "Hillary Clinton") %>% 
  mutate(pct_dem = candidatevotes / totalvotes)

trump <- election.2016.2 %>%
  filter(candidate == "Donald Trump") %>% 
  mutate(pct_rep = candidatevotes / totalvotes)

Now that we've made some progress, let's examine the data and see whether we are able to merge our nonspatial election data with our spatial county data.

In [7]:
tail(hillary)
tail(trump)

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,pct_dem
3153,2016,Wyoming,WY,Washakie,56043,President,Hillary Clinton,democrat,532,3814,20181011,0.13948610
3154,2016,Wyoming,WY,Weston,56045,President,Hillary Clinton,democrat,299,3526,20181011,0.08479864
3155,2016,Connecticut,NA,Statewide writein,NA,President,Hillary Clinton,democrat,NA,NA,20181011,NA
3156,2016,Maine,NA,Maine UOCAVA,NA,President,Hillary Clinton,democrat,3017,NA,20181011,NA
3157,2016,Alaska,NA,District 99,NA,President,Hillary Clinton,democrat,274,NA,20181011,NA
3158,2016,Rhode Island,NA,Federal Precinct,NA,President,Hillary Clinton,democrat,637,NA,20181011,NA


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,pct_rep
3153,2016,Wyoming,WY,Washakie,56043,President,Donald Trump,republican,2911,3814,20181011,0.7632407
3154,2016,Wyoming,WY,Weston,56045,President,Donald Trump,republican,3033,3526,20181011,0.8601815
3155,2016,Connecticut,NA,Statewide writein,NA,President,Donald Trump,republican,NA,NA,20181011,NA
3156,2016,Maine,NA,Maine UOCAVA,NA,President,Donald Trump,republican,648,NA,20181011,NA
3157,2016,Alaska,NA,District 99,NA,President,Donald Trump,republican,40,NA,20181011,NA
3158,2016,Rhode Island,NA,Federal Precinct,NA,President,Donald Trump,republican,53,NA,20181011,NA


We can see that the last four rows of both the Hillary and Trump datasets contain missing values. Because these would be complicated to include on our county map, given that there is no FIPS code for these regions, we will opt to remove the four problem entries. 

In [8]:
### Remove entries with NA FIPS
trump <- trump[which(!is.na(trump$FIPS)),]
hillary <- hillary[which(!is.na(hillary$FIPS)),]

Now that that's done, we appear to be ready to consolidate our data. Let's first extract only the data we need from these datasets. 

In [9]:
final.2016 <- data.frame(FIPS = hillary$FIPS, pct_hillary = hillary$pct_dem, pct_trump = trump$pct_rep)
glimpse(final.2016)

Observations: 3,154
Variables: 3
$ FIPS        <int> 1001, 1003, 1005, 1007, 1009, 1011, 1013, 1015, 1017, 101…
$ pct_hillary <dbl> 0.23769671, 0.19385601, 0.46527844, 0.21249575, 0.0842582…
$ pct_trump   <dbl> 0.7276659, 0.7654571, 0.5209667, 0.7640322, 0.8933484, 0.…


## Joining the Spatial and Non-Spatial Data
Now that we have cleaned our non-spatial data, we are ready to join it to our county shapefile. Let's first restructure the FIPS code on the county shapefile and then merge the data using sp's merge function to preserve the spatial format of the data.

In [10]:
fips <- as.integer(paste(us.counties@data$STATEFP, us.counties@data$COUNTYFP, sep = ""))
us.counties$FIPS <- fips

county.elections <- sp::merge(us.counties, final.2016, by = "FIPS")

## Writing a Shapefile
We are done! Now we simply have to write our newly created spatial dataset to a shapefile. Visualizations of this datset are not included in this due to the limited plotting capabilities of Jupyter notebook. 

In [21]:
writeOGR(county.elections,".", "county_level_election_results_2016", driver = "ESRI Shapefile")

Warning message in writeOGR(county.elections, ".", "county_level_election_results_2016", :
“Field names abbreviated for ESRI Shapefile driver”